# Project Notes

## Overview

I started with the Jupyter Notebook (Part 1) to develop the initial code. This code served as the foundation for Part 2 and has been further developed into the `train.py`, `predict.py`, `netcode.py`, and `dataload.py` scripts. 

## Development Process

### Part 1: Jupyter Notebook
- Initial code development was done in a Jupyter Notebook.
- Basic functionalities, including training and saving model weights, were implemented.

### Part 2: Script-Based Implementation
- The code evolved into separate Python scripts:
  - `train.py`: Handles training of the model.
  - `predict.py`: Used for making predictions with the trained model.
  - `netcode.py`: Contains the core neural network code.
  - `dataload.py`: Manages data loading and preprocessing.
- Several optimizations and improvements were made in these scripts compared to the original notebook implementation.

## Key Differences and Improvements

- **Model Saving and Loading**:
  - In Part 1 (Jupyter Notebook), only the model weights were saved.
  - In Part 2, a more comprehensive approach was adopted:
    - The entire model state, including architecture, optimizer state, and other relevant information, is saved.
    - This allows for complete restoration of the training state.

## Note

I did not backport the optimizations and improvements discovered in Part 2 to the original Jupyter Notebook. Therefore, the notebook in Part 1 does not include these enhancements. For example, the model saving process in Part 2 is significantly more comprehensive than in Part 1.


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Package Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import time
import copy
import torch.nn.functional as F
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook. The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = '../flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# Training data augmentation
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) # data normalization

# Test and valid data aren't augmented
test_transforms = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) # data normalization

valid_transforms = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) # data normalization

# Data loading
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

batch_size = 64

# Data batching
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4, pin_memory=True, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=4, pin_memory=True, shuffle=False)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=4, pin_memory=True, shuffle=True)

data_transforms = { 'train': train_transforms, 'test': test_transforms, 'valid': valid_transforms }
image_datasets = { 'train': train_data, 'test': test_data, 'valid': valid_data}
dataloaders = { 'train': trainloader, 'test': testloader, 'valid': validloader}

dataset_sizes = {
    'train': len(image_datasets['train']),
    'valid': len(image_datasets['valid']),
    'test': len(image_datasets['test'])
}

class_names = image_datasets['train'].classes

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
# create a label mapping from the category file
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Mapping directory names to indices
class_to_idx = image_datasets['train'].class_to_idx
idx_to_name = {v: cat_to_name[str(int(k))] for k, v in class_to_idx.items()}

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pretrained Network (I've decided to use the resnet18 net because it is fast and it works for me). 
# Tried others like VGG, MobileNetV2 but resnet18 seems to be very fast (good suitability for our data set of flowers so it seems)
model = models.resnet18(pretrained=True)

model.class_to_idx = class_to_idx
model.idx_to_name = idx_to_name

# Feedforward Classifier: change classification output layer according our num of output classes
num_classes = len(class_to_idx)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Only the classification layer should be trained
# gradients deactivation 
for param in model.parameters():
    param.requires_grad = False   

for param in model.fc.parameters():
        param.requires_grad = True
    
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# The lambda is needed because for the gradient activation (learning rate = 0.001)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# I override the learning rate of the default optimizer and integrate a StepLR strategy (because I wanted to test that too)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# print(model)

# you can load a previously saved model here
start_epoch = load_model(model, optimizer, 'saved_model.pth')
#start_epoch = 0
print(f"Continue with epoch: {start_epoch}")

# Training the network
def train_model(model, criterion, optimizer, scheduler, start_epoch=0, num_epochs=20):
    since = time.time()

    # lets deep copy the current model so we can return the model with the highest accuracy later on
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(start_epoch, num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # time measure for epoch
        epoch_since = time.time()
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            # Validation Loss and Accuracy
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc

                # if the model of the current epoch is better than a previous model, deep copy it
                best_model_wts = copy.deepcopy(model.state_dict())

        epoch_time_elapsed = time.time() - epoch_since
        print(f'Epoch {epoch} complete in {epoch_time_elapsed // 60:.0f}m {epoch_time_elapsed % 60:.0f}s')
        print()

    # Validation Loss and Accuracy
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # return the best (most accurate) model
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
# Training the network
model = train_model(model, criterion, optimizer, scheduler, start_epoch, num_epochs=start_epoch + 5)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# evaluate model function
def evaluate_model(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    total_loss = running_loss / len(dataloader.dataset)
    total_acc = running_corrects.double() / len(dataloader.dataset)

    print(f'Test Loss: {total_loss:.4f} Acc: {total_acc:.4f}')

# Testing Accuracy
evaluate_model(model, dataloaders['test'], criterion)

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Saving the model
model_save_path = 'saved_model.pth'

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': 25,  # optional: last epoch
    'loss': 0.5442,  # optional: last loss
    'class_to_idx': model.class_to_idx,
    'idx_to_name': model.idx_to_name
}, model_save_path)

print(f'Model saved to {model_save_path}')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# Loading checkpoints
def load_model(model, optimizer, model_path):
    
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # Optional: restore epoch and loss
    epoch = checkpoint['epoch']

    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_name = checkpoint['idx_to_name']
    
    return epoch + 1


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# Image Processing
def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    image = Image.open(image_path)

    # Scale the image to shortest side and keep aspect ration
    aspect_ratio = image.width / image.height
    if aspect_ratio > 1:        
        new_size = (int(aspect_ratio * 256), 256)
    else:        
        new_size = (256, int(256 / aspect_ratio))
    
    image = image.resize(new_size)

    # crop around the center
    width, height = image.size
    left = (width - 224) / 2
    top = (height - 224) / 2
    right = left + 224
    bottom = top + 224
    
    image = image.crop((left, top, right, bottom))

    # get a numpy array
    np_image = np.array(image)

    # normalize to 0-1 (because our net expects normalized values)
    np_image = np_image / 255.0

    # mean and deviation
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std

    # dimension ordering: color at first place
    np_image = np_image.transpose((2, 0, 1))

    return torch.from_numpy(np_image).float()

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing (revert to denormalized values)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
# Image Processing
test_image = process_image(test_dir + "/28/image_05230.jpg")
imshow(test_image)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# Class Prediction
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    loaded_img = process_image(image_path)
    loaded_img = loaded_img.to(device)  # don't forget to move the img to the proper cuda device

    model.eval()
    
    with torch.no_grad():
        loaded_img = loaded_img.unsqueeze(0)  # add batch dimension because model expects a batched input form
        output = model(loaded_img)
        probs, indices = torch.topk(output, topk)
        probs = probs.exp()

    # Convert indices to original folder names (which were numbers too by the way)
    idx_to_class = {v: k for k, v in model.class_to_idx.items()}
    classes = [idx_to_class[idx] for idx in indices[0].cpu().numpy()]

    return probs[0].cpu().numpy(), classes # return probs on the cpu, they don't need to be on the GPU 
    

In [ ]:
# Class Prediction
probs, classes = predict(test_dir + "/28/image_05230.jpg", model)
print(probs)
print(classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# Sanity Checking with matplotlib
def view_classify(image, ps, classes):
    ''' Function for viewing an image and it's predicted classes.
    '''    
    ps = ps.squeeze()

    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing (revert to denormalized values)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(image)
    ax1.axis('off')
    
    y_pos = np.arange(len(classes))
    ax2.barh(y_pos, ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(classes, size='small')
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()
    plt.show()

In [ ]:
# convert directory names (numbers) to names
# Mapping directory names to indices
class_to_idx = image_datasets['test'].class_to_idx
idx_to_name = {v: cat_to_name[str(int(k))] for k, v in class_to_idx.items()}

#print(class_to_idx)
#print()
#print(idx_to_name)

class_names = [idx_to_name[class_to_idx[idx]] for idx in classes]

test_image = process_image(test_dir + "/28/image_05230.jpg")
# Sanity Checking with matplotlib
view_classify(test_image, probs, class_names)